In [1]:
from model.data import *
d = Data()

Using TensorFlow backend.


In [2]:
# weights-01-0.02-0429-binary-patch.hdf5
train_result = h5py.File("./model/h5df_data/train_result-0429-binary-patch.h5", 'r')

d.show_image([train_result["result"][0, :], train_result["result"][1, :], train_result["result"][2, :]])
print(train_result["dice"][()])

interactive(children=(IntSlider(value=64, description='id', max=127), Output()), _dom_classes=('widget-interac…

0.8703509972045116
